In [61]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-4"

In [2]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [63]:
functions = [
  { 
    "name" : "multiply",
    "description": "multiplies a list of numbers to produce a result",
    "parameters": {
      "type" : "object",
      "properties": {
        "args": {
            "type":"array",
             "items" : {
                "type" : "number"
             }
         }
       } 
     
     }
    },
    { 
    "name" : "alternate_cases",
    "description": "converts alternate characters to upper and lower case",
    "parameters": {
      "type" : "object",
      "properties": {
        "input": {
            "type":"string",
         }
       } 
     }
    }  
]

    
def multiply(*args):
    result = 1
    for x in  args:
        result = result * x
    return result

def alternate_cases (input):
    i = 0
    rc = ""
    for c in input:
        c  = i % 2 == 0?c
            c = c.lower()
        else:
            c = c.upper()
        rc += c
        i+=1
    return rc

available_funcs = {
 "multiply": multiply,
 "alternate_cases": alternate_cases
}

    

In [66]:
print(list(filter(lambda x: x['name']=='multiply', functions))[0])

{'name': 'multiply', 'description': 'multiplies a list of numbers to produce a result', 'parameters': {'type': 'object', 'properties': {'args': {'type': 'array', 'items': {'type': 'number'}}}}}


In [47]:
messages = [
 {
     "role" :"system",
     "content": "respond to user input"
 },

    {
     "role" : "user",
     "content": "alternate cases in this sentence: 'hello I am Bob'"
 }
]

In [70]:
chat_response = chat_completion_request(messages, functions)
jsonR = chat_response.json()
mess = jsonR["choices"][0]["message"]
if mess['function_call'] is not None:
    funcName = mess['function_call']['name']
    toExecute = list(filter(lambda x: x['name']==funcName, functions))[0]
    args = json.loads(mess["function_call"]["arguments"])
    print(available_funcs[funcName](args['input']))

hElLo i aM BoB


In [14]:
print(openai.api_key)
